In [44]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [41]:
raw_data = pd.read_csv('adding_previous15_data.csv')

In [42]:
data = raw_data[raw_data['is_consumption'] == 1]

In [34]:
data.columns

Index(['county', 'is_business', 'product_type', 'target', 'is_consumption',
       'datetime', 'data_block_id', 'row_id', 'prediction_unit_id', 'date',
       'year', 'quarter', 'month', 'week', 'hour', 'day_of_year',
       'day_of_month', 'day_of_week', 'eic_count_client',
       'installed_capacity_client', 'date_client', 'temperature_h_mean',
       'dewpoint_h_mean', 'rain_h_mean', 'snowfall_h_mean',
       'surface_pressure_h_mean', 'cloudcover_total_h_mean',
       'cloudcover_low_h_mean', 'cloudcover_mid_h_mean',
       'cloudcover_high_h_mean', 'windspeed_10m_h_mean',
       'winddirection_10m_h_mean', 'shortwave_radiation_h_mean',
       'direct_solar_radiation_h_mean', 'diffuse_radiation_h_mean', 'hour_h',
       'hours_ahead_f_mean', 'temperature_f_mean', 'dewpoint_f_mean',
       'cloudcover_high_f_mean', 'cloudcover_low_f_mean',
       'cloudcover_mid_f_mean', 'cloudcover_total_f_mean',
       '10_metre_u_wind_component_f_mean', '10_metre_v_wind_component_f_mean',
       

In [51]:
columns_to_difference = [
    'temperature_h_mean', 'dewpoint_h_mean', 'rain_h_mean', 'snowfall_h_mean', 'surface_pressure_h_mean',
    'cloudcover_total_h_mean', 'cloudcover_low_h_mean', 'cloudcover_mid_h_mean', 'cloudcover_high_h_mean',
    'windspeed_10m_h_mean', 'winddirection_10m_h_mean', 'shortwave_radiation_h_mean',
    'direct_solar_radiation_h_mean', 'diffuse_radiation_h_mean','temperature_f_mean', 'dewpoint_f_mean',
    'cloudcover_high_f_mean', 'cloudcover_low_f_mean', 'cloudcover_mid_f_mean', 'cloudcover_total_f_mean',
    '10_metre_u_wind_component_f_mean', '10_metre_v_wind_component_f_mean', 'direct_solar_radiation_f_mean',
    'surface_solar_radiation_downwards_f_mean', 'snowfall_f_mean','target',
    'target_2_days_ago', 'target_3_days_ago',
    'target_4_days_ago', 'target_5_days_ago', 'target_6_days_ago',
    'target_7_days_ago', 'target_8_days_ago', 'target_9_days_ago',
    'target_10_days_ago', 'target_11_days_ago', 'target_12_days_ago',
    'target_13_days_ago', 'target_14_days_ago', 'target_15_days_ago'
    # Add the names of the columns you want to difference here
]
# Define the window size for differencing (e.g., 90 days)
window_size = 90

# Calculate the rolling mean for each specified column
for column in columns_to_difference:
    data[f'{column}_rolling_mean'] = data.groupby('prediction_unit_id')[column].transform(lambda x: x.rolling(window=window_size).mean())

# Calculate the differences and store them in new columns
for column in columns_to_difference:
    data[f'{column}_differenced'] = data[column] - data[f'{column}_rolling_mean']

# Drop the intermediate rolling mean columns if not needed
for column in columns_to_difference:
    data.drop(columns=[f'{column}_rolling_mean'], inplace=True)
    
data.dropna(inplace=True)


In [52]:
columns_to_difference.remove('target')
columns_drop = columns_to_difference+['date', 'year', 'quarter', 'month', 'week', 'hour', 'day_of_year','day_of_month', 'day_of_week', 'data_block_id', 'row_id','hour_h', 'hours_ahead_f_mean']
data.drop(columns=columns_drop, inplace=True)

In [54]:
data.columns

Index(['county', 'is_business', 'product_type', 'target', 'is_consumption',
       'datetime', 'prediction_unit_id', 'eic_count_client',
       'installed_capacity_client', 'date_client',
       'total_precipitation_f_mean', 'forecast_date_electricity',
       'euros_per_mwh_electricity', 'origin_date_electricity',
       'forecast_date_gas', 'lowest_price_per_mwh_gas',
       'highest_price_per_mwh_gas', 'origin_date_gas',
       'mean_price_per_mwh_gas', 'temperature_h_mean_differenced',
       'dewpoint_h_mean_differenced', 'rain_h_mean_differenced',
       'snowfall_h_mean_differenced', 'surface_pressure_h_mean_differenced',
       'cloudcover_total_h_mean_differenced',
       'cloudcover_low_h_mean_differenced',
       'cloudcover_mid_h_mean_differenced',
       'cloudcover_high_h_mean_differenced',
       'windspeed_10m_h_mean_differenced',
       'winddirection_10m_h_mean_differenced',
       'shortwave_radiation_h_mean_differenced',
       'direct_solar_radiation_h_mean_differe

In [55]:
data.to_csv('differenced.csv')